In [7]:
import pandas as pd

# 1. Load your aggregated card stats
cards_df = pd.read_csv("card_stats_royalapi.csv")


# 2. Build contingency table:

usage_ct = cards_df.pivot_table(
    index="Bracket",
    columns="CardName",
    values="UsagePct",
    aggfunc="mean",
    fill_value=0
)

# 3. Drop cards that have zero total usage across all brackets
usage_ct = usage_ct.loc[:, usage_ct.sum(axis=0) > 0]

print(usage_ct.shape)
print(usage_ct.head())


(4, 122)
CardName    Archer Queen  Archers  Arrows  Baby Dragon  Balloon  Bandit  \
Bracket                                                                   
All Ranked           4.0      2.0    27.0          4.5      9.0     6.0   
Ladder               2.0      4.5    27.0          4.5      6.0     5.0   
Top 1000             2.0      1.5    15.0          7.0      9.0     4.0   
Top 200              2.0      2.0    13.0          9.0      9.0     4.0   

CardName    Barbarian Barrel  Barbarians  Bats  Battle Healer  ...  Tornado  \
Bracket                                                        ...            
All Ranked              27.0         0.5   4.0            3.0  ...     10.0   
Ladder                   8.0         2.5   5.5            1.0  ...      4.0   
Top 1000                41.0         1.0   2.0            3.0  ...     14.0   
Top 200                 45.0         0.5   1.5            3.0  ...     13.0   

CardName    Tower Princess  Valkyrie  Vines  Wall Breakers  Witch

In [8]:
from scipy.stats import chi2_contingency

chi2, p, dof, expected = chi2_contingency(usage_ct.values)

print(f"Chi-square statistic: {chi2:.3f}")
print(f"p-value: {p:.5f}")
print(f"Degrees of freedom: {dof}")


Chi-square statistic: 425.039
p-value: 0.01364
Degrees of freedom: 363


In [10]:
import pandas as pd
import numpy as np
from scipy.stats import chi2_contingency

# 1. Load the aggregated card stats
cards_df = pd.read_csv("card_stats_royalapi.csv")


# 2. Pivot to get UsagePct and WinPct with:

usage = cards_df.pivot_table(
    index="Bracket",
    columns="CardName",
    values="UsagePct",
    aggfunc="mean"
)

winpct = cards_df.pivot_table(
    index="Bracket",
    columns="CardName",
    values="WinPct",
    aggfunc="mean"
)

# 3. Choose cards that are used (UsagePct > 0) in all brackets
cards_all_brackets = usage.columns[(usage > 0).all(axis=0)]

# 4. For each card, build a 2×4 contingency table:
n_base = 1000  #

chi2_results = {}  # card -> (chi2, p, dof)

for card in cards_all_brackets:
    u = usage[card].fillna(0)     # UsagePct per bracket
    w = winpct[card].fillna(0)    # WinPct per bracket

    # approximate counts
    uses = u * n_base / 100.0
    wins = uses * w / 100.0
    losses = uses - wins

    # skip cards with effectively zero total usage
    if uses.sum() <= 0:
        continue

    # 2×4 contingency table
    observed = np.vstack([wins.values, losses.values])

    # skip if some bracket has zero total (wins+losses = 0)
    if np.any(observed.sum(axis=0) == 0):
        continue

    chi2, p, dof, expected = chi2_contingency(observed)
    chi2_results[card] = (chi2, p, dof)

# 5. Inspect raw p-values (sorted)
sorted_results = sorted(chi2_results.items(), key=lambda x: x[1][1])
for card, (chi2, p, dof) in sorted_results[:10]:
    print(f"{card:20s}  chi2={chi2:.3f}  p={p:.4f}  dof={dof}")


Skeleton Barrel       chi2=8.332  p=0.0396  dof=3
Tower Princess        chi2=3.231  p=0.3573  dof=3
The Log               chi2=3.016  p=0.3891  dof=3
Hog Rider             chi2=1.467  p=0.6898  dof=3
Fireball              chi2=1.299  p=0.7295  dof=3
Goblin Gang           chi2=1.136  p=0.7683  dof=3
Arrows                chi2=1.129  p=0.7701  dof=3
Electro Wizard        chi2=1.111  p=0.7743  dof=3
Witch                 chi2=1.073  p=0.7837  dof=3
Mega Knight           chi2=0.850  p=0.8375  dof=3


In [12]:
alpha = 0.05
m = len(chi2_results)                 # number of cards tested
alpha_bonf = alpha / m               # Bonferroni threshold

significant_cards = [
    card for card, (chi2, p, dof) in chi2_results.items()
    if p < alpha_bonf
]

print(f"Number of cards tested: {m}")
print(f"Bonferroni alpha per test: {alpha_bonf:.6f}")
print("Cards with significant bracket differences in win rate:")



Number of cards tested: 113
Bonferroni alpha per test: 0.000442
Cards with significant bracket differences in win rate:
